In [7]:
!pip install transformers

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV  #finding best model
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')


In [9]:
import pandas as pd

df=pd.read_csv('/content/draft_50000.csv')
df['text'] = df['text'].str.lower().str.replace('[^\w\s]', '', regex=True)
df.dropna(subset=['text', 'class'], inplace=True)
df.head()

,Unnamed: 0,text,class
0,51297,get crush guy definitely way old laugh literal...,0.0
1,24705,go to july 2018i hope 8 month enough change mind,1.0
2,185969,want live anymore 23 right thinking end life p...,1.0
3,201675,every time get period want kill already depres...,1.0
4,52701,story incomplete similar story op case dad spe...,1.0


In [10]:
df["class"].value_counts()

class
1.0    2630
0.0    2611
Name: count, dtype: int64

In [12]:
from sklearn import preprocessing
data = preprocessing.LabelEncoder()
df["text"] = data.fit_transform(df["text"])
df.head()

,Unnamed: 0,text,class
0,51297,1673,0.0
1,24705,1852,1.0
2,185969,4974,1.0
3,201675,1206,1.0
4,52701,4226,1.0


In [13]:
import nltk # Import the nltk module first
nltk.download('punkt')
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english') # Use words() instead of word()
stopwords.extend(['.',',','!','?',';',':'])
#Remove stopwords from a given text
def remove_stopword(txt):
  # Use word_tokenize instead of word_tokenizer
  tokens=nltk.word_tokenize(str(txt)) # Convert txt to string before tokenization
  new_tokens=[token for token in tokens if token.lower() not in stopwords]
  return ' '.join(new_tokens)
df['text']  =df['text'].apply(remove_stopword)
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Unnamed: 0,text,class
0,51297,1673,0.0
1,24705,1852,1.0
2,185969,4974,1.0
3,201675,1206,1.0
4,52701,4226,1.0


In [14]:
# For BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

## Want DistilBERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [15]:
tokenized = df["text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(tokenized)

0            [101, 16785, 2509, 102]
1                   [101, 8784, 102]
2       [101, 4749, 2581, 2549, 102]
3             [101, 6036, 2575, 102]
4            [101, 29269, 2575, 102]
                    ...             
5236                [101, 5385, 102]
5237                [101, 6365, 102]
5238               [101, 14078, 102]
5239          [101, 2753, 2575, 102]
5240         [101, 4749, 26187, 102]
Name: text, Length: 5241, dtype: object


In [16]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [17]:
np.array(padded).shape

(5241, 5)

In [18]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5241, 5)

In [19]:
import torch

max_length = 512

# Assuming 'padded' is the NumPy array containing token IDs
input_ids = torch.tensor(padded)  # Convert padded to a PyTorch tensor
attention_mask = torch.tensor(attention_mask) # Convert to PyTorch tensor

# Truncate input_ids and attention_mask
input_ids = input_ids[:, :max_length]
attention_mask = attention_mask[:, :max_length]

# Proceed with the model inference
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [20]:
print(attention_mask.size())

torch.Size([5241, 5])


In [21]:
features = last_hidden_states[0][:,0,:].numpy()

In [22]:
c = df["text"]
#print(c)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(features,df["class"])

In [24]:
!pip install xgboost

In [25]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [26]:
import numpy as np
pred= model.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

         0.0       0.60      0.56      0.58       670
         1.0       0.57      0.60      0.58       641

    accuracy                           0.58      1311
   macro avg       0.58      0.58      0.58      1311
weighted avg       0.58      0.58      0.58      1311

